### ESCOM - IIA
#### FUNDAMENTOS DE INTELIGENCIA ARTIFICIAL - PROYECTO ML
#### Semestre 2023-2 - Grupo 4BV2
--- 
##### Autores:
- **Valdés Luis Eliot Fabián**

In [ ]:
# importamos la librerias necesarias
from ucimlrepo import fetch_ucirepo  # libreria de repositorios de datasets
import pandas as pd  # libreria para el manejo de dataframes

----
# PASO 1: Carga de datos
Sección inicial para obtener el dataset, describirlo de manera general usando estadisticas

In [ ]:
# definimos los datasets a utilizar
datasets = {
    "Eliot": 53, # Iris
    "Ethel": 17, # Breast Cancer
    "Leo": 878, # Cirrhosis
    "Adair": 109, # Wine
}
dataset = fetch_ucirepo(id=datasets["Ethel"]) # cargamos el dataset que deseemos utilizar

In [ ]:
# mostramos un pequeño resumen de lo que trata el dataset 
dataset.metadata.abstract

In [ ]:
# obtenemos los datos
X = dataset.data.features 
y = dataset.data.targets 

In [ ]:
# creamos un dataframe con los datos
df = pd.DataFrame(X, columns=dataset.data.feature_names)
# agregamos la columna target
df['target'] = y

# mostramos la cantidad de registros
print(f'Cantidad de registros: {len(df)}')

# imprimimos los primeros 5 registros
df.head()

In [ ]:
# mostramos los tipos de datos de cada columna
print(f"Tipos de datos (Metodo pandas):\n{df.dtypes}\n")

In [ ]:
# mostramos los tipos de datos usando los metodos de la librearia de repositorios
dataframe_tipos = pd.DataFrame({'Variable Name': dataset.variables['name'], 'Type': dataset.variables['type']})
print(f"Tipos de datos (Metodo ucimlrepo):\n{dataframe_tipos}\n")

In [ ]:
# por cada columna hacer lo siguiente 
# si el tipo de dato es numerico, mostrar la media, mediana, desviacion estandar, minimo y maximo
# si el tipo de dato es categorico, mostrar la cantidad de valores unicos

# limitamos la cantidad de columnas a 10
for col in df.columns[:10]:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        print(f"=> '{col}':")
        print(f"\t-Media: {df[col].mean()}")
        print(f"\t-Mediana: {df[col].median()}")
        print(f"\t-Desviacion estandar: {df[col].std()}")
        print(f"\t-Minimo: {df[col].min()}")
        print(f"\t-Maximo: {df[col].max()}")
    else:
        print(f"=> '{col}':")
        print(f"\t-Valores unicos: {df[col].unique()}")
        print(f"\t-Cantidad de valores unicos: {df[col].nunique()}")

----

# PASO 2: PREPROCESAMIENTO DE DATOS

Sección para preprocesar los datos, aquí separamos el dataset en los vectores de entrada X & de salida Y(las clases que son definidas con tipo de dato categorico)


In [ ]:
# funcion para separar cualquier dataframe en dos vectores (entrada y salida)
def separate_dataframe(df):
    vector_x = df.drop(df.columns[-1], axis=1) # obtencion de variables de entrada
    vector_y = df[df.columns[-1]] # obtencion de variable de salida (clase/target)
    return vector_x, vector_y

# separamos el dataframe en dos vectores, uno con las variables independientes y otro con la variable dependiente
vector_x, vector_y = separate_dataframe(df)

In [ ]:
# mostramos los primeros 5 registros del vector de entrada X
vector_x.head()

In [ ]:
# mostramos los primeros 5 registros del vector de salida Y
vector_y.head()

In [ ]:
# creamos una funcion que recibe como parametros el vector de entrada X y el vector de salida Y
def describe_categories(vector_x, vector_y):
    # agrupamos el vector de entrada X de acuerdo al vector de salida Y
    grouped = vector_x.groupby(vector_y)
    for name, group in grouped:
        print("\n",("="*50))
        print(f'Clase: {name}')
        print("="*50)
        # por cada grupo, accedemos a cada una de las columnas
        for col in group.columns:
            # validamos que el tipo de dato de la columna sea numerico y de ser el caso mosstramos las estadisticas
            if group[col].dtype == 'float64' or group[col].dtype == 'int64':
                print(f"=> '{col}':")
                print(f"\t-Media: {group[col].mean()}")
                print(f"\t-Mediana: {group[col].median()}")
                print(f"\t-Desviacion estandar: {group[col].std()}")
                print(f"\t-Minimo: {group[col].min()}")
                print(f"\t-Maximo: {group[col].max()}")
            else:
                print(f"=> '{col}':")
                print(f"\tValores unicos: {group[col].unique()}")
                print(f"\tCantidad de valores unicos: {group[col].nunique()}")            
        

describe_categories(vector_x, vector_y)

-----
# PASO 3: LIMPIEZA DE DATASET
Sección para limpiar el dataset para eliminar valores que no aportan al modelo, como los valores nulos o NaN.

In [ ]:
# juntamos los vectores de entrada y salida en un solo dataframe
df = vector_x.join(vector_y)
# mostramos la cantidad de registros
print(f'Cantidad de registros (Before cleaning): {len(df)}')

In [ ]:
# eliminamos los registros que tengan valores nulos en alguna de las columnas
df = df.dropna()
# mostramos la cantidad de registros
print(f'Cantidad de registros: (After cleaning) {len(df)}')

In [ ]:
# de nuevo separamos el dataframe en dos vectores, uno con las variables independientes y otro con la variable dependiente
vector_x, vector_y = separate_dataframe(df)

In [ ]:
# conservamos unicamente las columnas con tipo de dato Inter o Continuous
vector_x = vector_x.select_dtypes(include=['int64', 'float64'])
# mostramos las primeras 5 filas del vector de entrada
vector_x.head()

In [ ]:
print(f'Otra forma de limpiar el vector de entrada. Usa el dataset original y los tipos de datos del dataset pero no es tan eficiente como el metodo anterior ya que alginas columnas con tipo de dato numerico no son numericas en el dataset')
"""
for col_name in vector_x.columns:
    # verificar el tipo de dato de cada columna usando el dataset
    if col_name in dataset.variables['name'].values:
        # obtener el tipo de dato de la columna en el dataset
        col_type = dataset.variables['type'][dataset.variables['name'] == col_name].values[0]
        # eliminamos del vector_x aquellas columnas que no son numericas
        if col_type != 'Integer' and col_type != 'Continuous':
            vector_x = vector_x.drop(col_name, axis=1)
        else:        
            # parseamos las columnas a tipo de dato float64
            vector_x[col_name] = vector_x[col_name].astype('float64')                    

# mostramos las primeras 5 filas del vector de entrada
vector_x.head()                    
"""

-----
# PASO 4: IMPLEMENTACIÓN DE MODELOS DE ML Y EVALUACIÓN CON DIFERENTES MÉTRICAS
Sección para implementar los modelos Minima Distancia y KNN (K=1) para evaluar con los metodos: train-test split, k-fold cross validation y bootstrapping.

### MODELOS

#### - DESARROLLO DE MODELO MINIMA DISTANCIA

In [ ]:
def calcular_centroides(X, y):
    clases = set(y)
    centroides = {c: [0] * len(X[0]) for c in clases}
    contador = {c: 0 for c in clases}

    for xi, yi in zip(X, y):
        contador[yi] += 1
        for i in range(len(xi)):
            centroides[yi][i] += xi[i]

    for c in centroides:
        centroides[c] = [x / contador[c] for x in centroides[c]]
    
    return centroides

def clasificador_minima_distancia(X, centroides):
    predicciones = []
    for xi in X:
        distancias = {c: sum((xi[j] - centroides[c][j])**2 for j in range(len(xi))) for c in centroides}
        predicciones.append(min(distancias, key=distancias.get))
    return predicciones


#### DESARROLLO DE MODELO K NEAREST NEIGHBORS (K=1)

In [ ]:
def clasificador_knn(X_train, y_train, X_test):
    predicciones = []
    for xi in X_test:
        distancias = [sum((xi[j] - X_train[i][j])**2 for j in range(len(xi))) for i in range(len(X_train))]
        min_index = distancias.index(min(distancias))
        predicciones.append(y_train[min_index])
    return predicciones

### MÉTODOS DE VALIDACIÓN

#### - DESARROLLO DE MÉTODO TEST-TRAIN SPLIT (80-20)

In [ ]:
def entrenamiento_prueba(X, y, porcentaje_prueba):
    indice_corte = int(len(X) * (1 - porcentaje_prueba))
    X_train = X[:indice_corte]
    y_train = y[:indice_corte]
    X_test = X[indice_corte:]
    y_test = y[indice_corte:]
    return X_train, y_train, X_test, y_test

#### - DESARROLLO DE MÉTODO K-FOLD CROSS VALIDATION (K=5)

In [ ]:
def k_fold_cross_validation(X, y, k):
    tamaño_fold = len(X) // k
    for i in range(k):
        inicio = i * tamaño_fold
        fin = (i + 1) * tamaño_fold if i != k - 1 else len(X)
        X_train = X[:inicio] + X[fin:]
        y_train = y[:inicio] + y[fin:]
        X_test = X[inicio:fin]
        y_test = y[inicio:fin]
        yield X_train, y_train, X_test, y_test

#### - DESARROLLO DE MÉTODO BOOTSTRAPPING (B=100)

In [ ]:
import random

def bootstrap(X, y, n):
    for _ in range(n):
        indices = [random.randint(0, len(X) - 1) for _ in range(len(X))]
        X_train = [X[i] for i in indices]
        y_train = [y[i] for i in indices]
        X_test = [X[i] for i in range(len(X)) if i not in indices]
        y_test = [y[i] for i in range(len(X)) if i not in indices]
        yield X_train, y_train, X_test, y_test

### FUNCIONES AUXILIARES

#### - FUNCION PARA CALCULAR LA RELACIÓN EFIENCIA-ERROR DE UN MODELO

In [ ]:
def calcular_eficiencia_error(y_real, y_pred):
    correctos = sum(1 for real, pred in zip(y_real, y_pred) if real == pred)
    eficiencia = correctos / len(y_real)
    error = 1 - eficiencia
    return eficiencia, error

#### - FUNCION PARA TRANSFORMAR UN DATAFRAME EN UNA LISTA DE LISTAS

In [ ]:
def transformar_datos(vector_x, vector_y):
    X = vector_x.values.tolist() 
    # Transformar vector_y a una lista
    y = vector_y.values.tolist() 
    y = [item[0] for item in y] if isinstance(y[0], list) else y
    return X, y

- ### 4 => CONTENEDOR DE EJECUCIÓN DE MODELO: DISTANCIA MINIMA

In [ ]:
def run_min_distance_model(X, y):
    # ENTRENAMIENTO Y PRUEBA
    X_train, y_train, X_test, y_test = entrenamiento_prueba(X, y, porcentaje_prueba=0.2)
    # Calcular centroides y hacer predicciones
    centroides = calcular_centroides(X_train, y_train)
    predicciones = clasificador_minima_distancia(X_test, centroides)
    # Calcular eficiencia y error
    eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
    print(f"Minima distancia: eficiencia = {eficiencia}, error = {error} - Metodo de entrenamiento y prueba")
    
    # K-FOLD CROSS VALIDATION
    eficiencias = []
    errores = []
    k = 5  # Número de folds
    for X_train, y_train, X_test, y_test in k_fold_cross_validation(X, y, k):
        centroides = calcular_centroides(X_train, y_train)
        predicciones = clasificador_minima_distancia(X_test, centroides)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / k
    error_promedio = sum(errores) / k
    print(f"Minima distancia: eficiencia = {eficiencia_promedio}, error = {error_promedio} - K-fold cross validation")
    
    # BOOTSTRAP
    eficiencias = []
    errores = []
    n_iteraciones = 100  # Número de iteraciones de Bootstrap
    for X_train, y_train, X_test, y_test in bootstrap(X, y, n_iteraciones):
        centroides = calcular_centroides(X_train, y_train)
        predicciones = clasificador_minima_distancia(X_test, centroides)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / n_iteraciones
    error_promedio = sum(errores) / n_iteraciones
    print(f"Minima distancia: eficiencia = {eficiencia_promedio}, error = {error_promedio} - Bootstrap")      


- ### 5 => CONTENDOR DE EJECUCIÓN DE MODELO: KNN(K=1)

In [ ]:
def run_knn_model(X, y):
    # ENTRENAMIENTO Y PRUEBA
    # Dividir los datos
    X_train, y_train, X_test, y_test = entrenamiento_prueba(X, y, porcentaje_prueba=0.2)
    # Hacer predicciones
    predicciones = clasificador_knn(X_train, y_train, X_test)
    # Calcular eficiencia y error
    eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
    print(f"KNN: eficiencia = {eficiencia}, error = {error} - Metodo de entrenamiento y prueba")
    
    # K-FOLD CROSS VALIDATION
    eficiencias = []
    errores = []
    k = 5  # Número de folds
    for X_train, y_train, X_test, y_test in k_fold_cross_validation(X, y, k):
        predicciones = clasificador_knn(X_train, y_train, X_test)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / k
    error_promedio = sum(errores) / k
    print(f"KNN: eficiencia = {eficiencia_promedio}, error = {error_promedio} - K-fold cross validation")
    
    # BOOTSTRAP
    eficiencias = []
    errores = []
    n_iteraciones = 10  # Número de iteraciones de Bootstrap
    for X_train, y_train, X_test, y_test in bootstrap(X, y, n_iteraciones):
        predicciones = clasificador_knn(X_train, y_train, X_test)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / n_iteraciones
    error_promedio = sum(errores) / n_iteraciones
    print(f"KNN: eficiencia = {eficiencia_promedio}, error = {error_promedio} - Bootstrap")       

### EJECUCIÓN DE MODELOS

In [ ]:
# transformamos los vectores de entrada y salida a listas
X, y = transformar_datos(vector_x, vector_y)

-----
# PASO 4: EJECUCIÓN DE DISTANCIA MINIMA
- #### 4.A => DISTANCIA MINIMA - EVALUADO CON: TRAIN-TEST SPLIT
- #### 4.B => DISTANCIA MINIMA - EVALUADO CON: K FOLD CROSS VALIDATION
- #### 4.C => DISTANCIA MINIMA - EVALUADO CON: BOOTSTRAP

In [ ]:
# ejecutamos el modelo de distancia minima y validamos internamente con cada metodo
run_min_distance_model(X, y)

-----
# PASO 5: EJECUCIÓN DE KNN(K=1)
- #### 5.A => KNN - EVALUADO CON: TRAIN-TEST SPLIT
- #### 5.B => KNN - EVALUADO CON: K FOLD CROSS VALIDATION
- #### 5.C => KNN - EVALUADO CON: BOOTSTRAP

In [ ]:
# ejecutamos el modelo de KNN y validamos internamente con cada metodo
run_knn_model(X, y)